<a href="https://colab.research.google.com/github/subashmb1998/Project_HKS/blob/main/pending_payments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!rm spark-3.1.2-bin-hadoop2.7.tgz

# Install findspark
!pip install -q findspark

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

# Initialize findspark
import findspark
findspark.init()

# Verify the installation
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("colab").getOrCreate()
spark

In [ ]:
payment_pend = spark.read \
      .format("csv") \
      .option("header", "true") \
      .option("inferSchema", "true") \
      .load("/content/drive/MyDrive/Projects/HKS/Customer Wise Payment Due.csv")

In [ ]:
payment_pend.show()

+---------+---------------+----------------+---------------+-------------+
|WARD NAME|CUSTOMER NUMBER|   CUSTOMER NAME|BUILDING NUMBER|BUILDING TYPE|
+---------+---------------+----------------+---------------+-------------+
|1-PANAYOR| VIM-001-000001|Sahadevakrishnan|          1/296|        House|
|1-PANAYOR| VIM-001-000001|Sahadevakrishnan|          1/296|        House|
|1-PANAYOR| VIM-001-000004|    Krishankutty|          1/291|        House|
|1-PANAYOR| VIM-001-000004|    Krishankutty|          1/291|        House|
|1-PANAYOR| VIM-001-000004|    Krishankutty|          1/291|        House|
|1-PANAYOR| VIM-001-000004|    Krishankutty|          1/291|        House|
|1-PANAYOR| VIM-001-000004|    Krishankutty|          1/291|        House|
|1-PANAYOR| VIM-001-000005|          Sabira|          1/304|         Shop|
|1-PANAYOR| VIM-001-000005|          Sabira|          1/304|         Shop|
|1-PANAYOR| VIM-001-000005|          Sabira|          1/304|         Shop|
|1-PANAYOR| VIM-001-00000

In [ ]:
from pyspark.sql.functions import *
pending_count = payment_pend.select("CUSTOMER NUMBER","BUILDING NUMBER").groupBy("CUSTOMER NUMBER", "BUILDING NUMBER") \
    .agg(count("building number").alias("TOTAL_NO_OF_PENDING_PAYMENT"))


In [ ]:
pending_count.show()

+---------------+---------------+---------------------------+
|CUSTOMER NUMBER|BUILDING NUMBER|TOTAL_NO_OF_PENDING_PAYMENT|
+---------------+---------------+---------------------------+
| VIM-001-000218|          1/135|                          2|
| VIM-003-000447|          3/155|                          2|
| VIM-004-000044|         4/1283|                          7|
| VIM-004-000429|            4/0|                          2|
| VIM-004-000927|            4/0|                          1|
| VIM-005-000122|          5/168|                          3|
| VIM-005-000431|            5/0|                          1|
| VIM-005-000660|          5/814|                          2|
| VIM-007-000010|          7/762|                          3|
| VIM-007-000120|          7/000|                          1|
| VIM-007-000484|          7/670|                          1|
| VIM-007-000546|           7/52|                          1|
| VIM-007-000696|              7|                          1|
| VIM-00

In [ ]:
pending_count.repartition(1) \
    .write \
    .mode("overwrite") \
    .option("header", "true") \
    .format("csv") \
    .option("path", "/content/drive/MyDrive/Projects/HKS/Pending_Payment_final") \
    .save()